In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2
!nvcc --version
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

In [54]:
%%cu
#include <stdio.h> //wersja GPU
#include <iostream>
#include <math.h>
#include <chrono> 
#include <cuda.h>
#include <math_constants.h>
#include <thrust/copy.h>
#include <cuda_runtime.h>

using namespace std;

__device__ float funkcja(float x) {
    return (x*x + 5.2)/(sin(0.5+0.1*x*x));
}

__global__ void obliczProstokat(float* a, float c, float delta, int N){
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    float x = c + (float)idx * delta;
    if (idx < N){
        a[idx] = funkcja(x) + funkcja(x + delta);
    }
}

__host__ float metodaProstokatow(float a, float b, int N){
    float delta = (b - a)/ N;
    cudaError_t errorcode = cudaSuccess;
    int size = N * sizeof(float);
    float* tabHost = (float*) malloc(size);
    float* tabDevice;
    if ((errorcode = cudaMalloc ((void**)&tabDevice, size)) != cudaSuccess){
        cout << "cudaMalloc(): " << cudaGetErrorString(errorcode) << endl;
        exit(1);
    }
    int blockSize = 256;
    int blockNum = N / blockSize + (N % blockSize == 0 ? 0 : 1);
    obliczProstokat <<< blockNum, blockSize >>> (tabDevice, a, delta, N);
    if ((errorcode = cudaMemcpy(tabHost, tabDevice, sizeof(float) * N, cudaMemcpyDeviceToHost)) != cudaSuccess){
        cout << "cudaMemcpy(): " << cudaGetErrorString(errorcode) << endl;
        exit(1);
    }
    float wynik = 0.0;
    for (int i = 0; i < N; i++){
        wynik += tabHost[i];
    }
    wynik *= delta / 2.0;
    free(tabHost);
    cudaFree(tabDevice);
    return wynik;
}

__global__ void obliczTrapez(float* a, float c, float delta, int N){
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    float x = c + (float)idx * delta;
    if (idx < N){
        a[idx] = delta + funkcja(x + delta);
    }
}

__host__ float metodaTrapezow(float c, float d, int N){
    float delta = (d - c)/ N;
    cudaError_t errorcode = cudaSuccess;
    int size = N * sizeof(float);
    float* tabHost = (float*)malloc(size);
    float* tabDevice;
    if ((errorcode = cudaMalloc ((void**)&tabDevice, size)) != cudaSuccess){
        cout << "cudaMalloc(): " << cudaGetErrorString(errorcode) << endl;
        exit(1);
    }
    int blockSize = 256;
    int blockNum = N / blockSize + (N % blockSize == 0 ? 0 : 1);
    obliczTrapez <<< blockNum, blockSize >>> (tabDevice, c, delta, N);
    if ((errorcode = cudaMemcpy(tabHost, tabDevice, sizeof(float) * N, cudaMemcpyDeviceToHost)) != cudaSuccess){
        cout << "cudaMemcpy(): " << cudaGetErrorString(errorcode) << endl;
        exit(1);
    }
    float wynik = 0.0;
    for (int i = 0; i < N; i++){
        wynik += tabHost[i];
    }
    wynik *= delta;
    free(tabHost);
    cudaFree(tabDevice);
    return wynik;
}



__device__ void ustawRdz(int* n_start, int* n_end, int n, int blockNum, int blockWidth){
    int threadId = blockWidth * blockIdx.x + threadIdx.x;
    int nextThreadId = threadId + 1;
    int threads = blockWidth * blockNum;
    *n_start = (threadId * n)/threads;
    *n_end = (nextThreadId * n)/threads;
}

__device__ void obliczParab(int blockNum, int blockWidth, double a, double b, int n, float* result){
    *result = 0;
    float h = (b - a)/n;
    int idx_start;
    int idx_end;
    ustawRdz(&idx_start, &idx_end, n-1, blockNum, blockWidth);
    for (int i = idx_start; i < idx_end; i += 2) {
        *result += (funkcja(a  + h * (i - 1)) + 4 * funkcja(a + h * (i)) + funkcja(a + h * (i + 1))) * h / 3;
    }
}

__global__ void obliczSimpson(int blockNum, int blockWidth, float* result, float a, float b, int n){
    float wynik = 0;
    obliczParab(blockNum, blockWidth, a, b, n, &wynik);
    result[(blockIdx.x * blockWidth + threadIdx.x)] = wynik;
}

__host__ float metodaSimpsona(float a, float b, int n){
    const int blockNum = 32;
    const int blockWidth = 32;
    float tabHost[blockNum*blockWidth] = {0};
    float* tabDevice;
    cudaMalloc ((void**)&tabDevice, sizeof(float) * blockNum * blockWidth);
    obliczSimpson <<< blockNum, blockWidth >>> (blockNum, blockWidth, tabDevice, a, b, n );
    cudaThreadSynchronize();
    cudaMemcpy(tabHost, tabDevice, sizeof(float) * blockNum * blockWidth, cudaMemcpyDeviceToHost);
    float wynik = 0;
    for (int i = 0; i != blockNum *  blockWidth; ++i){
        wynik += tabHost[i];
    }
    cudaFree (tabDevice);
    return wynik;
}

int main(){
    float a = 1.2;
    float b = 2.2;
    int n = 100000000;
    float wynikProst = metodaProstokatow(a, b, n); //Metoda prostokatow
    cout << "Wynik dla metody Prostokatow: " << wynikProst << endl;
    float wynikTrap = metodaTrapezow(a, b, n); //Metoda trapezow
    cout << "Wynik dla metody Trapezow: " << wynikTrap << endl;
    float wynikSimp = metodaSimpsona(a, b, n); //Metoda simpsona
    cout << "Wynik dla metody Simpsona: " << wynikSimp << endl;
    return 0;
}

Wynik dla metody Prostokatow: 2.68435
Wynik dla metody Trapezow: 2.68435
Wynik dla metody Simpsona: 10.8002



In [53]:
%%cu
#include <stdio.h> //wersja CPU
#include <iostream>
#include <math.h>
#include <chrono> 
#include <math_constants.h>

using namespace std;

float funkcja(float x) {
    return (x*x + 5.2)/(sin(0.5+0.1*x*x));
}

float metodaProstokatow(float a, float b, int N){
    float delta = (b - a)/ N;
    float suma = 0, wynik;
		for (int i = 0; i < N; i++)	{
			suma += delta * funkcja(a + delta*(i + 0.5));
		}
		wynik = suma;
    return wynik;
}

float metodaTrapezow(float a, float b, int n)
{
    float wynik = 0;
    float delta = (b-a)/n;
    int i;
    for(i=1;i<=n-1;i++)
    {
        wynik += funkcja(a+i*delta);
    }
    wynik += (funkcja(a)+funkcja(b))/2;
    wynik *= delta;
    return wynik;
}
float metodaSimpsona(float a, float b, int n){
    float delta = (b - a) /n, s, wynik, x;
    wynik = 0;
    s = 0;
    for (int i=1; i < n; i++) {
      x = a + i*delta;
      s += funkcja(x - delta / 2);
      wynik += funkcja(x);
    }
    s += funkcja(b - delta / 2);
    wynik = (delta/6) * (funkcja(a) + funkcja(b) + 2*wynik + 4*s);
    return wynik;
}

int main(){
    float a = 1.2;
    float b = 2.2;
    int n = 100000000;
    float wynikProst = metodaProstokatow(a, b, n); //Metoda prostokatow
    cout << "Wynik dla metody Prostokatow: " << wynikProst << endl;
    float wynikTrap = metodaTrapezow(a, b, n); //Metoda trapezow
    cout << "Wynik dla metody Trapezow: " << wynikTrap << endl;
    float wynikSimp = metodaSimpsona(a, b, n); //Metoda simpsona
    cout << "Wynik dla metody Simpsona: " << wynikSimp << endl;
    return 0;
}

Wynik dla metody Prostokatow: 3.93707
Wynik dla metody Trapezow: 2.68435
Wynik dla metody Simpsona: 2.68435

